In [21]:
!pip install q keras==2.2.4
!pip install q scikit-learn==0.22.1
!pip install q tensorflow==2.0.0

     |████████████████████████████████| 317kB 4.4MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
  Found existing installation: Keras 2.4.3
    Uninstalling Keras-2.4.3:
      Successfully uninstalled Keras-2.4.3


     |████████████████████████████████| 7.1MB 3.3MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


     |████████████████████████████████| 86.3MB 36kB/s 
     |████████████████████████████████| 3.8MB 45.3MB/s 
     |████████████████████████████████| 450kB 40.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=6568783128a4b4cc905ff8553df6638c43715de98fbca0c3f7d6c5dd4d2bb7c6
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: t

In [22]:
import pandas as pd
import nltk as nltk
import sklearn
from sklearn import *
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
import tensorflow as tf
import keras
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer,SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk import tokenize
import numpy as np
import html
import re


import io
import contextlib

from IPython.display import clear_output
import math
pd.set_option(
    'display.max_colwidth', 0
)
np.random.seed(42)

In [23]:
nltk.download('subjectivity')
nltk.download('vader_lexicon')

[nltk_data] Downloading package subjectivity to /root/nltk_data...
[nltk_data]   Package subjectivity is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [24]:
csv_loc = '/content/drive/My Drive/LAMBDA/sh_data_clean.csv'
df = pd.read_csv(csv_loc,nrows=10000)


1. Score all comments obj/subj and compound
2. Do topic analysis for all topics


In [ ]:
df.describe(include="all")

In [26]:
sid = SentimentIntensityAnalyzer()

def preprocess_df(df):
    df = df[df['comment'].notna()]
    df['neg'], df['neu'], df['pos'], df["compound"], df["subjectivity"] = [np.nan, np.nan,np.nan,np.nan,np.nan]    
    return df

def add_sentiment(df):
    for i, row in df.iterrows():
        text = row["comment"]
        text = html.unescape(text)
        text =  re.sub('<[^<]+?>', '', text)
        # these are needed for 2020 data
        text = re.sub('href=" rel="nofollow">', '', text) 
        text = re.sub('>', '', text) 
        ## end of 2020 additions
        df.at[i,"comment"] = text
        ss = sid.polarity_scores(text)
        for k in ss:
            df.at[i,k] = ss[k]


def apply_subjobj(df):
  f = io.StringIO()
  from IPython.display import clear_output
  for i, row in df.iterrows():
      text = row["comment"]
      output = None
      with contextlib.redirect_stdout(f):
          try:
            nltk.sentiment.util.demo_sent_subjectivity(text)
          except:
            print('ooops')
      output = f.getvalue()
      val = output.split("\n")
      df["subjectivity"][i] = val[-2]
  return df

        
pipeline = Pipeline(
    [
        ('vect', CountVectorizer(analyzer='word')),
        ('tfidf', TfidfTransformer()),    
    ]
)     

In [27]:
df = preprocess_df(df)
add_sentiment(df)
df = apply_subjobj(df)
df.head()

,created_at,username,comment,story_title,comment_id,neg,neu,pos,compound,subjectivity
0,2020-08-25T15:10:23.000Z,bradstewart,"Nobody in the semiconductor industry will put up with anyone in the supply chain making a 600% margin.Large Nvidia chips might not be able to move quickly, but all of the MCU, WiFi, etc chips would migrate to cheaper fabs like SMIC. Which reduces TSMC's revenue, which reduces their ability to invest in new processes. Cents (and even fractions of cents) matter to those chips.",‘Better Yield on 5nm Than 7nm’: TSMC Update on Defect Rates for N5,24272132,0.000,0.892,0.108,0.7469,subj
1,2020-08-25T15:10:17.000Z,nabla9,You can't just create new fab with money.,‘Better Yield on 5nm Than 7nm’: TSMC Update on Defect Rates for N5,24272131,0.168,0.555,0.277,0.2928,subj
2,2020-08-25T15:10:08.000Z,emteycz,they actually consider themselves to be the legitimate government of ChinaThat's not the case since 1991.,‘Better Yield on 5nm Than 7nm’: TSMC Update on Defect Rates for N5,24272130,0.000,1.000,0.000,0.0000,obj
3,2020-08-25T15:10:08.000Z,dgellow,"Let’s say competing app stores won’t be used. Then why is it an issue for Apple to allow them, given that they know they would still be the default and have the advantage? Apple is in this situation at where control the platform completely, and could have the default store. But somehow that’s not enough for them to consider opening the app market a little bit.Using that reasoning, they are in the best position to actually open to competition.","Apple ordered to not block Epic’s Unreal Engine, Fortnite to stay off App Store",24272129,0.000,0.903,0.097,0.7964,subj
4,2020-08-25T15:10:01.000Z,James_Henry,There is a shortage of truly random easily sampled noise though.,Challenge to scientists: does your ten-year-old code still run?,24272128,0.140,0.490,0.371,0.5106,subj


In [28]:
X_train, X_test = train_test_split(df)

In [29]:
df.to_csv("tenk.csv")

In [30]:
scored = X_train.loc[X_train['compound'] != 0]

In [32]:
X = pipeline.fit_transform(scored["comment"])

## Topics 

In [33]:
nltk.download('wordnet')
nltk.download('stopwords')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
from nltk.corpus import wordnet as wn
import random
import spacy
spacy.load('en')
from spacy.lang.en import English
from nltk.stem.wordnet import WordNetLemmatizer
en_stop = set(nltk.corpus.stopwords.words('english'))

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens


def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens    

In [35]:

parser = English()
texts = []

df['tokens'] = [[]] * df.shape[0]

def add_tokens(df):
    sid = SentimentIntensityAnalyzer()
    for i, row in df.iterrows():
        text = row["comment"]
        tokens = prepare_text_for_lda(text)
        tok = list(tokens)
        df.at[i,"tokens"] = tokens

In [36]:
add_tokens(df)

In [37]:
from gensim import corpora
import pickle
tokenslist = df["tokens"].tolist()
dictionary = corpora.Dictionary(tokenslist)
corpus = [dictionary.doc2bow(text) for text in tokenslist]
# save if needed
#pickle.dump(corpus, open('corpus.pkl', 'wb'))
#dictionary.save('dictionary.gensim')

In [38]:
import gensim
from gensim import models
# trying diff
NUM_TOPICS = 100
commentslist = df.comment.to_list()

### LDA

In [39]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topicslda = ldamodel.print_topics()
for topic in topicslda:
    print(topic)

(37, '0.126*"make" + 0.100*"sense" + 0.034*"normal" + 0.028*"large" + 0.025*"anyway" + 0.024*"think" + 0.023*"familiar" + 0.023*"statement" + 0.022*"usage" + 0.021*"covid"')
(54, '0.081*"china" + 0.070*"match" + 0.062*"python" + 0.056*"facebook" + 0.050*"binary" + 0.042*"define" + 0.039*"pattern" + 0.036*"unique" + 0.029*"balance" + 0.028*"relative"')
(20, '0.056*"spend" + 0.046*"million" + 0.041*"accept" + 0.040*"effort" + 0.032*"effect" + 0.032*"startup" + 0.031*"suggest" + 0.026*"algorithm" + 0.026*"order" + 0.021*"marketing"')
(91, '0.086*"marriage" + 0.072*"quality" + 0.041*"lower" + 0.039*"willing" + 0.036*"improve" + 0.031*"successful" + 0.030*"structure" + 0.025*"success" + 0.021*"finding" + 0.020*"specifically"')
(94, '0.085*"message" + 0.078*"miss" + 0.058*"ubuntu" + 0.056*"host" + 0.051*"imply" + 0.032*"truly" + 0.025*"crazy" + 0.024*"drop" + 0.023*"stream" + 0.023*"ridiculous"')
(85, '0.073*"knowledge" + 0.072*"process" + 0.050*"analysis" + 0.045*"exact" + 0.037*"disagree" 

In [ ]:
# Try 20 topics but lets use 100
# ldamodel2 = gensim.models.ldamodel.LdaModel(corpus, num_topics = 20, id2word=dictionary, passes=20)
# ldamodel2.save('model_lda20topics.gensim')

In [40]:
# LSI model is faster but topics are less on point 

# The transformations are standard Python objects, typically initialized by means of a training corpus:
tfidf = models.TfidfModel(corpus)
# From now on, tfidf is treated as a read-only object that can be used to convert any vector from the old representation 
# (bag-of-words integer counts) to the new representation (TfIdf real-valued weights):
corpus_tfidf = tfidf[corpus]
lsi_model2 = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=NUM_TOPICS)  # initialize an LSI transformation
corpus_lsi2 = lsi_model[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
print(lsi_model.print_topic(1))

NameError: ignored

### LSI
Not using this but just for exploration

In [41]:
#tfidf = models.TfidfModel(corpus)
# From now on, tfidf is treated as a read-only object that can be used to convert any vector from the old representation 
# (bag-of-words integer counts) to the new representation (TfIdf real-valued weights):
#corpus_tfidf = tfidf[corpus]
lsi_model2 = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=20)  # initialize an LSI transformation
corpus_lsi2 = lsi_model2[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi


In [42]:
# this is how we access LSI topics
for doc, as_text in zip(corpus_lsi2, commentslist):    
    srt = sorted(doc, key=lambda sublist: abs(sublist[1]))
    # reversing after sorting 
    srt.reverse()
    if len(srt) > 1:
      if (abs(srt[0][1]) > 0.5):
        print(srt[0], as_text)

(9, -0.5316512470175082) Full article here: 
(1, -0.687864451844027) Good answer, thanks
(1, -0.6525970329089936) Thanks friend!
(1, -0.9716098972220233) Thanks!
(3, 0.5034441557257551) Well, I don't comment on Reddit without reading the article, but I notice that, on contentious topics, by the time I've read the article there are 2000 comments already and there's not much point posting a comment after that.  If you want somebody to read your comment, you have to post on something nobody cares about, or post REALLY fast.
(2, -0.5937980575329135) I’ve been running PCs and Macs in my house for over 2 decades. I have not once had a problem with viruses.Most of my problems have been with restrictions that Apple has put on me in their Mac and iOS product lines. Apple requiring Sign In with Apple (when signin with Google/Facebook are implemented) where you can hide your email address. That’s not possible with multiple app stores.Yes it is. You, as a consumer can choose to stay in the Apple a

[(0, 0.08205038701546617), (62, 0.059695892364185314), (8, 0.05458417342154055)] Nobody in the semiconductor industry will put up with anyone in the supply chain making a 600% margin.Large Nvidia chips might not be able to move quickly, but all of the MCU, WiFi, etc chips would migrate to cheaper fabs like SMIC. Which reduces TSMC's revenue, which reduces their ability to invest in new processes. Cents (and even fractions of cents) matter to those chips.


In [ ]:
# corpus_lda = ldamodel[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi
# for doc, as_text in zip(corpus_lda, commentslist):
#     srt = sorted(doc, key=lambda sublist: abs(sublist[1]))
#     srt.reverse()  
#     if len(srt) > 1:
#       if (abs(srt[0][1]) > 0.6):
#         print(srt[0], as_text)

### Populating the dataframe with topics

In [43]:
# LETS USE LDA IT IS MORE ACCURATE
import warnings
warnings.filterwarnings("ignore")

corpus_lda = ldamodel[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

topz = []
for doc, as_text in zip(corpus_lda, commentslist):
    srt = sorted(doc, key=lambda sublist: abs(sublist[1]))
    srt.reverse()
    #topiks = srt
    topiks = srt[:3]
    topz.append([topiks,as_text])

# not working WHY?
# for i, row in df.iterrows():
#   text = row["comment"]
#   for doc, as_text in zip(corpus_lda, row["comment"]):
#     srt = sorted(doc, key=lambda sublist: abs(sublist[1]))
#     srt.reverse()
#     topiks = srt
#     if len(srt) > 3:
#       topiks = srt[:3]
#     df.at[i, 'topics'] = pd.Series(topiks)
#     df.at[i, 'topics_nums'] = pd.Series([i[0] for i in topiks])

In [44]:
# Filling in 1 topic
topz_single = []
for topss in topz:
  if (len(topss[0]) > 0): 
    topz_single.append(topss[0][0])  
  else:
    topz_single.append([0,0])  

idx = 0
df['topics_num'] = np.nan
for top in topz_single:
  df["topics_num"][idx] = top[0]
  idx = idx+1

In [45]:
# Filling in multiple topics
idx = 0
df['topics_nums'] = [[]] * df.shape[0]
df['topics'] = [[]] * df.shape[0]
for top in topz_single:
  df["topics"][idx] = pd.Series(top[0])
  df["topics_nums"][idx] = pd.Series([i[0] for i in topiks])
  idx = idx+1

In [46]:
# Not using just some demo
df['topics'] = [[]] * df.shape[0]
lst = list(topz[1][0])
df.loc[df['comment'] == topz[1][1], "topics"] = pd.Series(lst)

In [47]:
df.to_csv("filled_10k.csv")

In [48]:
subjectives = df.loc[df["subjectivity"]=="subj"]

In [49]:
negativesubj = subjectives.loc[df["compound"]<0]

In [50]:
negativesubj.shape

(2343, 14)

In [51]:
user = pd.DataFrame()
user["username"] = df['username']
user["topics_all"], user["compound"] = [[[]] * user.shape[0], np.nan]

df_topics = df.groupby('username')['topics_num'].apply(list)
df_compounds = df.groupby('username')['compound'].apply(list)


In [52]:
user = user[:-1]

In [53]:
for i, row in user.iterrows():
  lst = list(df_topics[row["username"]])
  lst_comp = list(df_compounds[row["username"]])
  comp = sum(lst_comp) / len(lst_comp)
  user["topics_all"][i] = lst
  user["compound"][i] = comp

In [54]:
user.loc[user["compound"]<-0.5]

,username,topics_all,compound
24,9nGQluzmnq3M,"[44.0, nan]",-0.50145
40,technofiend,[4.0],-0.89220
47,PascLeRasc,[13.0],-0.51870
72,jjoonathan,"[34.0, nan, nan]",-0.65650
74,juskrey,[60.0],-0.93770
...,...,...,...
9937,aljungberg,[nan],-0.72120
9952,8fingerlouie,[nan],-0.65620
9965,sparkie,"[nan, nan]",-0.64890
9974,dmortin,[nan],-0.83400


In [55]:
# all negative users and their topics:
vc = user.loc[user["compound"]<-0.5].topics_all.apply(tuple).value_counts()
worst_topics =[]
for key in vc.keys():
  if vc[key] > 5:
    print(f"\nMentioned {vc[key]} times")
    print(ldamodel.print_topic(int(key[0])))
    #worst_topics.append(key)


Mentioned 7 times
0.069*"move" + 0.041*"living" + 0.034*"area" + 0.032*"regular" + 0.032*"weight" + 0.029*"ignore" + 0.026*"patient" + 0.025*"deliver" + 0.021*"propose" + 0.020*"oracle"

Mentioned 6 times


ValueError: ignored

In [57]:
topnegtopics = negativesubj["topics_num"].value_counts()
# print(ldamodel.print_topic(78))
# print(ldamodel.print_topic(75))
print(ldamodel.print_topic(69))


0.305*"years" + 0.160*"first" + 0.064*"couple" + 0.047*"second" + 0.030*"light" + 0.023*"longer" + 0.018*"code" + 0.017*"necessary" + 0.016*"double" + 0.015*"beginning"


In [58]:
negativesubj.head()

,created_at,username,comment,story_title,comment_id,neg,neu,pos,compound,subjectivity,tokens,topics_num,topics_nums,topics
9,2020-08-25T15:09:13.000Z,908B64B197,"That's a problem (paying for training and loosing their doctors) for foreign countries, not the US.",Top colleges should select randomly from a pool of ‘good enough’ (2005),24272122,0.162,0.838,0.000,-0.4019,subj,"[problem, pay, training, loose, doctor, foreign, country]",7.0,0 0 1 80 dtype: int64,[]
21,2020-08-25T15:08:20.000Z,nabla9,"Large volume manufacturing deals are negotiated sometimes years before, TSMC had no idea how much demand there was or how badly Intel fails when AMD, NVDA and Apple bought the capacity.New GIGAFAB takes time to build.",‘Better Yield on 5nm Than 7nm’: TSMC Update on Defect Rates for N5,24272108,0.231,0.769,0.000,-0.8225,subj,"[large, volume, manufacturing, deal, negotiate, sometimes, years, demand, badly, intel, fail, apple, buy, capacity, gigafab, take, build]",96.0,0 0 1 80 dtype: int64,[]
24,2020-08-25T15:08:03.000Z,9nGQluzmnq3M,"I have no dog in the fight, but if this comment is actually wrong (as implied by the downvotes), it'd be nice to hear why.",Expert predicts a major hurricane hitting Houston would be “America’s Chernobyl”,24272105,0.242,0.640,0.118,-0.4310,subj,"[fight, comment, actually, wrong, imply, downvotes]",44.0,0 0 1 80 dtype: int64,[]
29,2020-08-25T15:06:57.000Z,me_me_me,"The 'government' is not an independent actor - its soft power. And its especially soft in US. This whole thing with 'executive power', look at me I am doing things - its all smoke and mirrors.Obama promised to end wars close down Guantanamo bay and in the ended up keeping up all war campaigns and on top of that he started droning people around the world.The 'big boys' of the industry have enough fingers in the pie(s) to have their say in things that would affect them.And independence of Taiwan is definitely one. Hong Kong fell without a peep from the U.S.Because it was mostly Chinese businessmen window to the world, is fall of independent HK really that big of a deal for US or UK?",‘Better Yield on 5nm Than 7nm’: TSMC Update on Defect Rates for N5,24272090,0.057,0.889,0.054,-0.3612,subj,"[government, independent, actor, power, especially, whole, thing, executive, power, things, smoke, mirror, obama, promise, close, guantanamo, end, keeping, campaign, start, droning, people, around, world, industry, enough, finger, pie(s, things, would, affect, independence, taiwan, definitely, without, u.s.because, mostly, chinese, businessmen, window, world, independent, really]",21.0,0 0 1 80 dtype: int64,[]
34,2020-08-25T15:06:32.000Z,thdrdt,"""you visit someone else’s server, and therefore they get data about your visit""I don't think the average person knows this. A lot of people even have no clue about internet. So there is no consent most of the time. And we, the developers, just let the logs running.""with kindle, you’re using your own device and there’s no expectation that amazon will be snooping""Well I would absolutely have this expectation. I expect a device that is connected to the internet snooping on me. Then there is the Amazon brand. I absolutely don't trust them so I expect them to snoop in me.But to be clear: I absolutely hate that my privacy is gone. I use all kinds of blockers to disable tracking and I also agree with jacquesm snooping is wrong. But I still think his point is too black and white and therefore unfair.",Kindle Collects a Surprisingly Large Amount of Data,24272085,0.130,0.802,0.068,-0.8853,subj,"[visit, someone, server, therefore, visit""i, think, average, person, know, people, internet, consent, developer, running, ""with, kindle, using, device, expectation, amazon, snooping""well, would, absolutely, expectation, expect, device, connect, internet, snoop, amazon, brand, absolutely, trust, expect, snoop, clear, absolutely, privacy, kind, blocker, disable, tracking, agree, jacquesm, snoop, wrong, still, think, point, black, white, th

In [ ]:
## OVERNIGHT:

df = preprocess_df(df)
add_sentiment(df)
df = apply_subjobj(df)

from nltk.corpus import wordnet as wn
import random
import spacy
spacy.load('en')
from spacy.lang.en import English
from nltk.stem.wordnet import WordNetLemmatizer
en_stop = set(nltk.corpus.stopwords.words('english'))

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens


def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens    


parser = English()
texts = []

df['tokens'] = [[]] * df.shape[0]

def add_tokens(df):
    sid = SentimentIntensityAnalyzer()
    for i, row in df.iterrows():
        text = row["comment"]
        tokens = prepare_text_for_lda(text)
        tok = list(tokens)
        df.at[i,"tokens"] = tokens    
add_tokens(df)        

from gensim import corpora
import pickle
tokenslist = df["tokens"].tolist()
dictionary = corpora.Dictionary(tokenslist)
corpus = [dictionary.doc2bow(text) for text in tokenslist]

pickle.dump(corpus, open('/content/drive/My Drive/Colab-Notebooks/Sentim/corpus.pkl', 'wb'))
dictionary.save('/content/drive/My Drive/Colab-Notebooks/Sentim/dictionary.gensim')

import gensim
from gensim import models
# trying diff
NUM_TOPICS = 100
commentslist = df.comment.to_list()

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('/content/drive/My Drive/Colab-Notebooks/model5.gensim')
topicslda = ldamodel.print_topics()



import warnings
warnings.filterwarnings("ignore")

corpus_lda = ldamodel[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

topz = []
for doc, as_text in zip(corpus_lda, commentslist):
    srt = sorted(doc, key=lambda sublist: abs(sublist[1]))
    srt.reverse()
    #topiks = srt
    topiks = srt[:3]
    topz.append([topiks,as_text])

# Filling in 1 topic
topz_single = []
for topss in topz:
  if (len(topss[0]) > 0): 
    topz_single.append(topss[0][0])  
  else:
    topz_single.append([0,0])  

idx = 0
df['topics_num'] = np.nan
for top in topz_single:
  df["topics_num"][idx] = top[0]
  idx = idx+1    

# Filling in multiple topics
idx = 0
df['topics_nums'] = [[]] * df.shape[0]
df['topics'] = [[]] * df.shape[0]
for top in topz_single:
  df["topics"][idx] = pd.Series(top[0])
  df["topics_nums"][idx] = pd.Series([i[0] for i in topiks])
  idx = idx+1  

df.to_csv("/content/drive/My Drive/Colab-Notebooks/Sentim/filled_1M.csv")  

In [59]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 3.4MB/s 
     |████████████████████████████████| 552kB 23.6MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=28cc1bff2ac666f3e0b131e6f35c3ed7ae9035cb572beeb992aad7ce87fa8469
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=e34a45c995c6f74cc27e144ae3cd2ee0b93bd989a4d6e060677adf5bf4c04ba8
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [60]:
import pyLDAvis.gensim

lda_disp = pyLDAvis.gensim.prepare(ldamodel,corpus,dictionary)
pyLDAvis.display(lda_disp)

### Emergency try

In [7]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [8]:
nltk.download('subjectivity')

[nltk_data] Downloading package subjectivity to /root/nltk_data...
[nltk_data]   Unzipping corpora/subjectivity.zip.


True

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [12]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [14]:
df = pd.read_csv(csv_loc,nrows=1000)

sid = SentimentIntensityAnalyzer()

def preprocess_df(df):
    df = df[df['comment'].notna()]
    df['neg'], df['neu'], df['pos'], df["compound"], df["subjectivity"] = [np.nan, np.nan,np.nan,np.nan,np.nan]    
    return df

def add_sentiment(df):
    for i, row in df.iterrows():
        text = row["comment"]
        text = html.unescape(text)
        text =  re.sub('<[^<]+?>', '', text)
        # these are needed for 2020 data
        text = re.sub('href=" rel="nofollow">', '', text) 
        text = re.sub('>', '', text) 
        ## end of 2020 additions
        df.at[i,"comment"] = text
        ss = sid.polarity_scores(text)
        for k in ss:
            df.at[i,k] = ss[k]


def apply_subjobj(df):
  f = io.StringIO()
  from IPython.display import clear_output
  for i, row in df.iterrows():
      text = row["comment"]
      output = None
      with contextlib.redirect_stdout(f):
          try:
            nltk.sentiment.util.demo_sent_subjectivity(text)
          except:
            print('ooops')
      output = f.getvalue()
      val = output.split("\n")
      df["subjectivity"][i] = val[-2]
  return df


df = preprocess_df(df)
add_sentiment(df)
df = apply_subjobj(df)

from nltk.corpus import wordnet as wn
import random
import spacy
spacy.load('en')
from spacy.lang.en import English
from nltk.stem.wordnet import WordNetLemmatizer
en_stop = set(nltk.corpus.stopwords.words('english'))

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens


def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens    


parser = English()
texts = []

df['tokens'] = [[]] * df.shape[0]

def add_tokens(df):
    sid = SentimentIntensityAnalyzer()
    for i, row in df.iterrows():
        text = row["comment"]
        tokens = prepare_text_for_lda(text)
        tok = list(tokens)
        df.at[i,"tokens"] = tokens    
add_tokens(df)        

from gensim import corpora
import pickle
tokenslist = df["tokens"].tolist()
dictionary = corpora.Dictionary(tokenslist)
corpus = [dictionary.doc2bow(text) for text in tokenslist]

# pickle.dump(corpus, open('/content/drive/My Drive/Colab-Notebooks/Sentim/corpus.pkl', 'wb'))
# dictionary.save('/content/drive/My Drive/Colab-Notebooks/Sentim/dictionary.gensim')

import gensim
from gensim import models
# trying diff
NUM_TOPICS = 100
commentslist = df.comment.to_list()

ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
# ldamodel.save('/content/drive/My Drive/Colab-Notebooks/model5.gensim')
topicslda = ldamodel.print_topics()



import warnings
warnings.filterwarnings("ignore")

tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

corpus_lda = ldamodel[corpus_tfidf]  # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

topz = []
for doc, as_text in zip(corpus_lda, commentslist):
    srt = sorted(doc, key=lambda sublist: abs(sublist[1]))
    srt.reverse()
    #topiks = srt
    topiks = srt[:3]
    topz.append([topiks,as_text])

...
# Filling in 1 topic
topz_single = []
for topss in topz:
  if (len(topss[0]) > 0): 
    topz_single.append(topss[0][0])  
  else:
    topz_single.append([0,0])  

idx = 0
df['topics_num'] = np.nan
for top in topz_single:
  df["topics_num"][idx] = top[0]
  idx = idx+1    

# Filling in multiple topics
idx = 0
df['topics_nums'] = [[]] * df.shape[0]
df['topics'] = [[]] * df.shape[0]
for top in topz_single:
  df["topics"][idx] = pd.Series(top[0])
  df["topics_nums"][idx] = pd.Series([i[0] for i in topiks])
  idx = idx+1  

# df.to_csv("/content/drive/My Drive/Colab-Notebooks/Sentim/filled_1M.csv")

In [16]:
subjectives = df.loc[df["subjectivity"]=="subj"]
negativesubj = subjectives.loc[df["compound"]<0]
user = pd.DataFrame()
user["username"] = df['username']
user["topics_all"], user["compound"] = [[[]] * user.shape[0], np.nan]

df_topics = df.groupby('username')['topics_num'].apply(list)
df_compounds = df.groupby('username')['compound'].apply(list)
user = user[:-1]
for i, row in user.iterrows():
  lst = list(df_topics[row["username"]])
  lst_comp = list(df_compounds[row["username"]])
  comp = sum(lst_comp) / len(lst_comp)
  user["topics_all"][i] = lst
  user["compound"][i] = comp

# demo
# topnegtopics = negativesubj["topics_num"].value_counts()
# print(ldamodel.print_topic(78))
# print(ldamodel.print_topic(75))
# print(ldamodel.print_topic(67))

# value counts on topics
vc = user.loc[user["compound"]<-0.5].topics_all.apply(tuple).value_counts()
for key in vc.keys():
  if vc[key] > 5:
    print(f"\nMentioned {vc[key]} times")
    print(ldamodel.print_topic(int(key[0])))


In [17]:
df.head()

,created_at,username,comment,story_title,comment_id,neg,neu,pos,compound,subjectivity,tokens,topics_num,topics_nums,topics
0,2020-08-25T15:10:23.000Z,bradstewart,"Nobody in the semiconductor industry will put up with anyone in the supply chain making a 600% margin.Large Nvidia chips might not be able to move quickly, but all of the MCU, WiFi, etc chips would migrate to cheaper fabs like SMIC. Which reduces TSMC's revenue, which reduces their ability to invest in new processes. Cents (and even fractions of cents) matter to those chips.",‘Better Yield on 5nm Than 7nm’: TSMC Update on Defect Rates for N5,24272132,0.000,0.892,0.108,0.7469,subj,"[nobody, semiconductor, industry, anyone, supply, chain, making, margin, large, nvidia, chips, might, quickly, chips, would, migrate, cheap, reduce, revenue, reduce, ability, invest, process, cent, fraction, cent, matter, chips]",0.0,0 44 dtype: int64,0 0 dtype: int64
1,2020-08-25T15:10:17.000Z,nabla9,You can't just create new fab with money.,‘Better Yield on 5nm Than 7nm’: TSMC Update on Defect Rates for N5,24272131,0.168,0.555,0.277,0.2928,subj,"[create, money]",98.0,0 44 dtype: int64,0 98 dtype: int64
2,2020-08-25T15:10:08.000Z,emteycz,they actually consider themselves to be the legitimate government of ChinaThat's not the case since 1991.,‘Better Yield on 5nm Than 7nm’: TSMC Update on Defect Rates for N5,24272130,0.000,1.000,0.000,0.0000,obj,"[actually, consider, legitimate, government, chinathat, since]",61.0,0 44 dtype: int64,0 61 dtype: int64
3,2020-08-25T15:10:08.000Z,dgellow,"Let’s say competing app stores won’t be used. Then why is it an issue for Apple to allow them, given that they know they would still be the default and have the advantage? Apple is in this situation at where control the platform completely, and could have the default store. But somehow that’s not enough for them to consider opening the app market a little bit.Using that reasoning, they are in the best position to actually open to competition.","Apple ordered to not block Epic’s Unreal Engine, Fortnite to stay off App Store",24272129,0.000,0.903,0.097,0.7964,subj,"[compete, store, issue, apple, allow, given, would, still, default, advantage, apple, situation, control, platform, completely, could, default, store, somehow, enough, consider, opening, market, little, using, reasoning, position, actually, competition]",67.0,0 44 dtype: int64,0 67 dtype: int64
4,2020-08-25T15:10:01.000Z,James_Henry,There is a shortage of truly random easily sampled noise though.,Challenge to scientists: does your ten-year-old code still run?,24272128,0.140,0.490,0.371,0.5106,subj,"[shortage, truly, random, easily, sample, noise, though]",1.0,0 44 dtype: int64,0 1 dtype: int64


In [18]:
user.head()

,username,topics_all,compound
0,bradstewart,[0.0],0.74690
1,nabla9,"[98.0, 69.0]",-0.26485
2,emteycz,[61.0],0.00000
3,dgellow,"[67.0, nan, nan, nan, nan]",0.50242
4,James_Henry,[1.0],0.51060


In [19]:
df.to_csv("/content/drive/My Drive/LAMBDA/hn_sample_nlp.csv")

In [20]:
user.to_csv("/content/drive/My Drive/LAMBDA/hn_sample_user_nlp.csv")